In [1]:
# %%bash
# cd notebooks
# git clone -b dev/#24/20201107/osugi https://github.com/ex-hasuolab/deeplearning-set.git

In [1]:
# !pip install tensorflow-gpu==2.2.0

In [49]:
# modelsのパス
models_path = '/home/esaka/models'
# deeplearning-setのtoolsまでのパス（docker-composeのvolume設定でマウントしておく）
# deeplearning_tools_path = '/home/esaka/notebooks/deeplearning-set'
# dataディレクトリのパス
# train_data_path = '/home/esaka/notebooks/ducky_data/'
# test_data_path = "/home/esaka/notebooks/ducky_data/"
# 予測したモデルをエクスポートする為のパス
# trained_model_output_path = '/home/esaka/notebooks/trained_model'

In [50]:
# # deeplearning-setにパスを通して、いろいろとimport出来るようにしておく
# import sys
# sys.path.append(deeplearning_tools_path)

In [19]:
import matplotlib
import matplotlib.pyplot as plt
import os
import shutil
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
%matplotlib inline
# deeplearnng-setデータローダ
# from tools.dataloader import Dataloader
# # 画像・BBOXプロット用関数
# from tools.render import plot_detections
%load_ext autoreload
import cv2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


## 画像サイズの確認

# model_build test

# Data用意

## trainとevalに分ける

In [5]:
# trainとevalに分ける
split_module_path = "/home/esaka/notebooks/preprocessing/partition_dataset.py"
datafolder_path = "/home/esaka/notebooks/data2/train/trimed"
!python {split_module_path} -x -i {datafolder_path} -r 0.1

## tfrecord化

In [6]:
# trainとevalに分ける
# ここでlabel mapがおかしい場合は弾いてくれる
generate_tfrecode__module_path = "/home/esaka/notebooks/preprocessing/generate_tfrecord.py"
datafolder_path = "/home/esaka/notebooks/data2/train/trimed/train"
annotation_path = "/home/esaka/notebooks/data2/train/trimed/annotations"
!python {generate_tfrecode__module_path}  -x {datafolder_path} -l {annotation_path}/label_map.pbtxt -o {annotation_path}/train.tfrecord

datafolder_path = "/home/esaka/notebooks/data2/train/trimed/test"
!python {generate_tfrecode__module_path}  -x {datafolder_path} -l {annotation_path}/label_map.pbtxt -o {annotation_path}/test.tfrecord

Successfully created the TFRecord file: /home/esaka/notebooks/data2/train/trimed/annotations/train.tfrecord
Successfully created the TFRecord file: /home/esaka/notebooks/data2/train/trimed/annotations/test.tfrecord


# モデルダウンロード

In [4]:
##change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    "ssd_v1_fpn_640x640_coco17_tpu-8": {
        "model_name": "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8",
        "base_pipeline_file" : "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config",
        "pretrained_checkpoint": "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz",
        "batch_size" : 1
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 1
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 1
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 1
    },
        'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 1
    }
}

In [5]:
#in this tutorial we implement the lightweight, smallest state of the art efficientdet model
#if you want to scale up tot larger efficientdet models you will likely need more compute!
chosen_model = 'ssd_v1_fpn_640x640_coco17_tpu-8'
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

In [6]:
folder_name = "my_ssd_trimedv1"

In [7]:
import tarfile
%mkdir /home/esaka/notebooks/{folder_name}
%cd /home/esaka/notebooks/{folder_name}
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

mkdir: cannot create directory ‘/home/esaka/notebooks/my_ssd_trimedv1’: File exists
/home/esaka/notebooks/my_ssd_trimedv1
--2020-11-19 06:35:56--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 216.58.199.240
Connecting to download.tensorflow.org (download.tensorflow.org)|216.58.199.240|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M  9.98MB/s    in 22s     

2020-11-19 06:36:18 (10.9 MB/s) - ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [244817203/244817203]

--2020-11-19 06:36:20--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.git

In [9]:
train_record_fname = "/home/esaka/notebooks/data2/train/trimed_v1/annotations/train.tfrecord"
test_record_fname = "/home/esaka/notebooks/data2/train/trimed_v1/annotations/test.tfrecord"
label_map_pbtxt_fname = '/home/esaka/notebooks/data2/train/trimed_v1/annotations/label_map.pbtxt'

In [10]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
print(num_classes)

2


In [11]:
pipeline_fname = "/home/esaka/notebooks/" + folder_name + "/" + base_pipeline_file
fine_tune_checkpoint = "/home/esaka/notebooks/" + folder_name + "/" + "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0"

In [12]:
num_steps = 100000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 5000 #Perform evaluation after so many steps

In [13]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file
import re

%cd /home/esaka/notebooks/{folder_name}
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

/home/esaka/notebooks/my_ssd_trimedv1
writing custom configuration file


In [14]:
pipeline_file = "/home/esaka/notebooks/" + folder_name + "/pipeline_file.config"
model_dir = "/home/esaka/notebooks/" + folder_name + "/training/"
TRAIN_MODULE_PATH = "/home/esaka/models/research/object_detection/model_main_tf2.py"

In [ ]:
!python {TRAIN_MODULE_PATH} \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I1119 06:36:46.983520 140618125211456 mirrored_strategy.py:341] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 100000
I1119 06:36:46.989045 140618125211456 config_util.py:552] Maybe overwriting train_steps: 100000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1119 06:36:46.989105 140618125211456 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Reading unweighted datasets: ['/home/esaka/notebooks/data2/train/trimed_v1/annotations/train.tfrecord']
I1119 06:36:47.112149 140618125211456 dataset_builder.py:148] Reading unweighted datasets: ['/home/esaka/notebooks/data2/train/trimed_v1/annotations/train.tfrecord']
INFO:tensorflow:Reading record datasets for input file: ['/home/esaka/notebooks/data2/train/trimed_v1/annotations/train.tfrecord']
I1119 06:36:47.113020 140618125

## tensorboardで確認

In [1]:
folder_name = "my_ssd"
model_dir = "/home/esaka/notebooks/" + folder_name + "/training/"

In [5]:
%reload_ext tensorboard
%tensorboard --logdir={model_dir} --bind_all --port 6006

## Modelをエクスポート

In [22]:
EXPORT_MODULE_PATH = "/home/esaka/models/research/object_detection/exporter_main_v2.py"

In [23]:
exported_model_dir = "/home/esaka/notebooks/" + folder_name + "/exported_model"

In [24]:
!python {EXPORT_MODULE_PATH} \
    --input_type image_tensor \
    --pipeline_config_path={pipeline_file} \
    --trained_checkpoint_dir={model_dir} \
    --output_directory {exported_model_dir}

W1118 20:45:46.457790 139629846468416 save_impl.py:78] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x7efd9f124c50>, because it is not built.
INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efc203ce710>, TensorSpec(shape=(None, 80, 80, 512), dtype=tf.float32, name='image_features/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efc203ce6a0>, TensorSpec(shape=(None, 40, 40, 1024), dtype=tf.float32, name='image_features/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efc203ce6d8>, TensorSpec(shape=(None, 20, 20, 2048), dtype=tf.float32, name='image_features/2/1'))], True), {}).
I1118 20:45:56.619814 139629846468416 def_function.py:1038] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7efc203ce710>, TensorSpec(shape=

## Modelをインポート

In [18]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [19]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: the file path to the image

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def get_keypoint_tuples(eval_config):
    """Return a tuple list of keypoint edges from the eval config.

    Args:
    eval_config: an eval config containing the keypoint edges

    Returns:
    a list of edge tuples, each in the format (start, end)
    """
    tuple_list = []
    kp_list = eval_config.keypoint_edge
    for edge in kp_list:
        tuple_list.append((edge.start, edge.end))
    return tuple_list

In [27]:
import os
import glob

In [23]:
folder_name

'my_ssd'

In [24]:
pipeline_config = "/home/esaka/notebooks/" + folder_name + "/exported_model/pipeline.config"
model_dir = "/home/esaka/notebooks/" + folder_name +"/exported_model/checkpoint"

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(model_dir, 'ckpt-0')).expect_partial()

def get_model_detection_function(model):
    """Get a tf.function for detection."""

    @tf.function
    def detect_fn(image):
        """Detect objects in image."""

        image, shapes = model.preprocess(image)
        prediction_dict = model.predict(image, shapes)
        detections = model.postprocess(prediction_dict, shapes)

        return detections, prediction_dict, tf.reshape(shapes, [-1])

    return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [25]:
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
i = 9

image_dir = '/home/esaka/notebooks/data2/train/trimed_v2/test'
filenames = glob.glob(image_dir + "/*.png")
image_np = load_image_into_numpy_array(filenames[i])

# Things to try:
# Flip horizontally
# image_np = np.fliplr(image_np).copy()

# Convert image to grayscale
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

# Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if 'detection_keypoints' in detections:
    keypoints = detections['detection_keypoints'][0].numpy()
    keypoint_scores = detections['detection_keypoint_scores'][0].numpy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'][0].numpy(),
    (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
    detections['detection_scores'][0].numpy(),
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=100,
    min_score_thresh=0.15,
    agnostic_mode=False,
    keypoints=keypoints,
    keypoint_scores=keypoint_scores,
    keypoint_edges=get_keypoint_tuples(configs['eval_config']))

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()

# 1枚の画像に対して結果を描写する

In [ ]:
image_id = 0
input_tensor = tf.convert_to_tensor(test_image_list[image_id], dtype=tf.float32)
img = input_tensor.numpy()[0]
detections = detect(input_tensor)

In [ ]:
def plot_bbox(bbox, img, width = 650, height = 650):   
    y1 = int(bbox[0] * height)
    x1 = int(bbox[1] * width)
    y2 = int(bbox[2] * height)
    x2 = int(bbox[3] * width)
    
    color_pallet = [(255,0,0), (0,255,0), (0,0,255), (255,255,0), (255,0,255), (0,255,255)]
    index, = np.where(class_list[n].numpy()[i].astype(int) == 1)
    color = color_pallet[index[0]]    
    img = cv2.rectangle(img, (x1,y1),(x2,y2),color, 5)
    
    return img

In [ ]:
detections.keys()

In [ ]:
detections["detection_classes"]

In [ ]:
thre = 0.3
object_indexes, = np.where(detections['detection_scores'][0].numpy() > thre)
print("{} object detected !!".format(len(object_indexes)))

for object_index in object_indexes:
    target_bbox = detections['detection_boxes'][0].numpy()[object_index]
    img = plot_bbox(target_bbox, img)

plt.figure(figsize=(10,20))
plt.imshow(img)

In [ ]:
# スコアの分布
plt.plot(detections['detection_scores'][0].numpy())
plt.ylim(0,1)

In [ ]:
# 評価指標mAPの計算
# def draw_map(true , predict):
#     hight = 255
#     width = 255
#     n = 0
#     Num_of_Hit = 0
#     Precision = 0
#     Ni = 0
#     AP = []
#     for i in range(width):
#         for j in range(hight):
#             if (i >= predict[1] and i < predict[3]) and (j >= predict[0] and j < predict[2]):
#                 n = n + 1   #モデルが顔だと認識した画素数を入れる
#                 if (i >= true[1] and i < true[3]) and (j >= true[0] and j < true[2]):
#                     Num_of_Hit = Num_of_Hit + 1   #モデルが顔だと認識した画素の内、正解だった画素数を入れる
#                     Precision = Precision + (Num_of_Hit / n)
# #                     Ni = Ni + Num_of_Hit
#                     Ni = Num_of_Hit
#                     APi = Precision / Ni
#                     AP.append(APi)
#     AP = np.array(AP) #下でmean関数を使う為にarray化する
#     mAP = np.mean(AP)
#     print("mAP:", mAP)
#     print("Num:", Num_of_Hit)
#     print(AP.shape)

In [ ]:
# git hubより
# detectionsの理解について
# from object_detection.utils import label_map_util
# category_index = label_map_util.create_category_index_from_labelmap(path_to_labels)
# for idx, box in enumerate(output_dict['detection_boxes'][0]):
#     if output_dict['detection_scores'][0][idx] > treshold:
#         class_name = category_index[int(output_dict['detection_classes'][0][idx])]['name']